In [1]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 316.9/316.9 MB 3.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.0-py2.py3-none-any.whl size=317425345 sha256=84e1d0775ce2aaf270c1146c59f2703007892f25827de83fc012b0e9c94911fe
  Stored in directory: /root/.cache/pip/wheels/41/4e/10/c2cf2467f71c678cfc8a6b9ac9241e5e44a01940da8fbb17fc
Successfully built pyspark


In [2]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [3]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"



In [4]:
# 2. Create a temporary view of the DataFrame.
spark.sparkContext.addFile(url)
df = spark.read.csv(SparkFiles.get("home_sales_revised.csv"), sep=",", header=True)
df.show()
df.createOrReplaceTempView('home_sales')

+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [24]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

spark.sql("SELECT ROUND(AVG(price), 2) AS AveragePrice,YEAR(date) AS YearBuilt FROM home_sales WHERE \
           bedrooms=4 \
           GROUP BY YEAR(date) \
           ORDER BY YEAR(date) DESC").show()

+------------+---------+
|AveragePrice|YearBuilt|
+------------+---------+
|   296363.88|     2022|
|   301819.44|     2021|
|   298353.78|     2020|
|    300263.7|     2019|
+------------+---------+



In [25]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

spark.sql("SELECT ROUND(AVG(price), 2) AS AveragePrice,YEAR(date) AS YearBuilt FROM home_sales WHERE \
           bedrooms=3 and \
            bathrooms=3 \
            GROUP BY YEAR(date) \
            ORDER BY YEAR(date) DESC").show()

+------------+---------+
|AveragePrice|YearBuilt|
+------------+---------+
|   292725.69|     2022|
|   294211.46|     2021|
|   294204.16|     2020|
|   287287.82|     2019|
+------------+---------+



In [26]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

spark.sql("SELECT ROUND(AVG(price), 2) AS AveragePrice,YEAR(date) AS YearBuilt FROM home_sales WHERE \
            bedrooms=3 and \
            bathrooms=3 and \
            floors=2 and \
            sqft_living>=2000 \
            GROUP BY YEAR(date) \
            ORDER BY YEAR(date) DESC").show()

+------------+---------+
|AveragePrice|YearBuilt|
+------------+---------+
|   290242.99|     2022|
|   296330.96|     2021|
|   292289.09|     2020|
|   289859.14|     2019|
+------------+---------+



In [27]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()

query = """SELECT view, ROUND(AVG(price), 2) AS AveragePrice FROM home_sales
            GROUP BY view
            HAVING AVG(price) >= 350000
            ORDER BY view DESC
        """
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))

+----+------------+
|view|AveragePrice|
+----+------------+
|  99|  1061201.42|
|  98|  1053739.33|
|  97|  1129040.15|
|  96|  1017815.92|
|  95|   1054325.6|
|  94|   1033536.2|
|  93|  1026006.06|
|  92|   970402.55|
|  91|  1137372.73|
|  90|  1062654.16|
|  89|  1107839.15|
|  88|  1031719.35|
|  87|   1072285.2|
|  86|  1070444.25|
|  85|  1056336.74|
|  84|  1117233.13|
|  83|  1033965.93|
|  82|   1063498.0|
|  81|  1053472.79|
|  80|   991767.38|
+----+------------+
only showing top 20 rows

--- 0.8665130138397217 seconds ---


In [29]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [30]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [31]:
# 9. Using the cached data, run the query that filters out the view ratings with average price
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

query = """SELECT view, ROUND(AVG(price), 2) AS AveragePrice FROM home_sales
            GROUP BY view
            HAVING AVG(price) >= 350000
            ORDER BY view desc
        """
spark.sql(query).show()

print("--- %s seconds ---" % (time.time() - start_time))


+----+------------+
|view|AveragePrice|
+----+------------+
|  99|  1061201.42|
|  98|  1053739.33|
|  97|  1129040.15|
|  96|  1017815.92|
|  95|   1054325.6|
|  94|   1033536.2|
|  93|  1026006.06|
|  92|   970402.55|
|  91|  1137372.73|
|  90|  1062654.16|
|  89|  1107839.15|
|  88|  1031719.35|
|  87|   1072285.2|
|  86|  1070444.25|
|  85|  1056336.74|
|  84|  1117233.13|
|  83|  1033965.93|
|  82|   1063498.0|
|  81|  1053472.79|
|  80|   991767.38|
+----+------------+
only showing top 20 rows

--- 0.5367162227630615 seconds ---


In [32]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data
df.write.partitionBy("date_built").mode("overwrite").parquet("home_sales_partitioned")

In [33]:
# 11. Read the parquet formatted data.
p_df_p=spark.read.parquet('home_sales_partitioned')

In [34]:
# 12. Create a temporary table for the parquet data.
p_df_p.createOrReplaceTempView('p_home_sales_partitioned')

In [35]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000
# with the parquet DataFrame. Round your average to two decimal places.
# Determine the runtime and compare it to the cached version.

start_time = time.time()
query = """SELECT view, ROUND(AVG(price), 2) AS AVERAGE_PRICE FROM p_home_sales_partitioned
            GROUP BY view
            HAVING AVG(price) >= 350000
            ORDER BY view desc
        """
spark.sql(query).show()


print("--- %s seconds ---" % (time.time() - start_time))

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
|  80|    991767.38|
+----+-------------+
only showing top 20 rows

--- 1.0306816101074219 seconds ---


In [36]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [37]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached('home_sales')

False